In [61]:
import pandas as pd
import constants as c
from importlib import reload

reload(c)

def clean_postcode_name(postcode):
    return(postcode.split(', ')[0].split(' ')[0])


def geo_update_sort_drop(df, geo, Y):
    df1 = df[df['RATIO_FROM_TO'] == 1.0]
    df2 = df[(df['RATIO_FROM_TO'] != 1.0) & (df['RATIO_FROM_TO'] >= 0.0)]
    df2 = df2.sort_values(by='RATIO_FROM_TO', ascending=False)
    df2 = df2.drop_duplicates(subset= geo + '_NAME_'+ Y, keep='first')
    return pd.concat([df1,df2])

### Building Postcode to LGA table

In [62]:
df = pd.read_csv(c.raw_data_file_lga, skiprows=10, skipfooter=10)[1:].rename(columns={"LGA (EN)": 'POA'})
df = df.melt(id_vars='POA', var_name='LGA', value_name='Rented Households')

df = df[df['Rented Households'] > 0]

df = df[df['LGA'] != 'Total']
df = df[~df['LGA'].str.startswith('Migratory')]

df['Postcode'] = df['POA'].apply(clean_postcode_name)
df = df[['Postcode', 'LGA', 'Rented Households']].reset_index(drop=True)

postcode_totals = df.groupby('Postcode').sum().reset_index().rename(columns={'Rented Households': 'Postcode Totals'})
lga_totals = df.groupby('LGA').sum().reset_index().rename(columns={'Rented Households': 'LGA Totals'})

df = df.merge(postcode_totals, on='Postcode', how='left')
df = df.merge(lga_totals, on='LGA', how='left')

df['Postcode Fraction'] = df['Rented Households']/df['Postcode Totals']
df['LGA Fraction'] = df['Rented Households']/df['LGA Totals']

df.sort_values(by=['Postcode', 'LGA']).to_csv(c.processed_data_dir + 'POA_LGA_Rented_Weightings.csv', index = False)

C:\Users\jack.m\AppData\Local\Temp\ipykernel_11912\3012478653.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(c.raw_data_file_lga, skiprows=10, skipfooter=10)[1:].rename(columns={"LGA (EN)": 'POA'})


#### LGA Correspondence files

In [63]:
lga_2016_2021 = pd.read_csv(c.raw_lga_2016_2021)
lga_2021_2022 = pd.read_csv(c.raw_lga_2021_2022)

lga_2021_2022 = geo_update_sort_drop(lga_2021_2022, 'LGA', '2021')
lga_2016_2021 = geo_update_sort_drop(lga_2016_2021, 'LGA', '2016')

lga_2021_2022.rename({'RATIO_FROM_TO': 'RATIO_FROM_2021_TO_2022'}, axis='columns', inplace=True)
lga_2016_2021.rename({'RATIO_FROM_TO': 'RATIO_FROM_2016_TO_2021'}, axis='columns', inplace=True)

lga_2021_2022[['LGA_CODE_2021', 'LGA_NAME_2021', 'LGA_CODE_2022', 'LGA_NAME_2022',
       'RATIO_FROM_2021_TO_2022']].to_csv(c.processed_data_dir + 'LGA_2021_To_2022.csv', index=False)
lga_2016_2021[['LGA_CODE_2016', 'LGA_NAME_2016', 'LGA_CODE_2021', 'LGA_NAME_2021',
       'RATIO_FROM_2016_TO_2021']].to_csv(c.processed_data_dir + 'LGA_2016_To_2021.csv', index=False)

### Building Postcode to SED table

In [64]:
df = pd.read_csv(c.raw_data_file_sed, skiprows=10, skipfooter=10)[1:].rename(columns={"SED (EN)": 'POA'})
df = df.melt(id_vars='POA', var_name='SED', value_name='Rented Households')

df = df[df['Rented Households'] > 0]

df = df[df['SED'] != 'Total']
df = df[~df['SED'].str.startswith('Migratory')]

df['Postcode'] = df['POA'].apply(clean_postcode_name)
df = df[['Postcode', 'SED', 'Rented Households']].reset_index(drop=True)

postcode_totals = df.groupby('Postcode').sum().reset_index().rename(columns={'Rented Households': 'Postcode Totals'})
sed_totals = df.groupby('SED').sum().reset_index().rename(columns={'Rented Households': 'SED Totals'})

df = df.merge(postcode_totals, on='Postcode', how='left')
df = df.merge(sed_totals, on='SED', how='left')

df['Postcode Fraction'] = df['Rented Households']/df['Postcode Totals']
df['SED Fraction'] = df['Rented Households']/df['SED Totals']

df.sort_values(by=['Postcode', 'SED']).to_csv(c.processed_data_dir + 'POA_SED_Rented_Weightings.csv', index=False)

C:\Users\jack.m\AppData\Local\Temp\ipykernel_11912\3223932162.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(c.raw_data_file_sed, skiprows=10, skipfooter=10)[1:].rename(columns={"SED (EN)": 'POA'})


#### SED Correspondence files

In [65]:
sed_2016_2021 = pd.read_csv(c.raw_sed_2016_2021)
sed_2021_2022 = pd.read_csv(c.raw_sed_2021_2022)

sed_2021_2022 = geo_update_sort_drop(sed_2021_2022, 'SED', '2021')
sed_2016_2021 = geo_update_sort_drop(sed_2016_2021, 'SED', '2016')

sed_2021_2022.rename({'RATIO_FROM_TO': 'RATIO_FROM_2021_TO_2022'}, axis='columns', inplace=True)
sed_2016_2021.rename({'RATIO_FROM_TO': 'RATIO_FROM_2016_TO_2021'}, axis='columns', inplace=True)

sed_2021_2022[['SED_CODE_2021', 'SED_NAME_2021', 'SED_CODE_2022', 'SED_NAME_2022',
       'RATIO_FROM_2021_TO_2022']].to_csv(c.processed_data_dir + 'SED_2021_To_2022.csv', index=False)
sed_2016_2021[['SED_CODE_2016', 'SED_NAME_2016', 'SED_CODE_2021', 'SED_NAME_2021',
       'RATIO_FROM_2016_TO_2021']].to_csv(c.processed_data_dir + 'SED_2016_To_2021.csv', index=False)